<a href="https://colab.research.google.com/github/Lada496/CrewAI-Projects/blob/main/Capstone_Project_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [6]:
!pip install 'crewai[tools]' -q
!pip install youtube-transcript-api -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 1.1.6 requires openai<3.0.0,>=1.109.1, but you have openai 1.83.0 which is incompatible.


In [7]:
!pip install langchain_openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.14.0 which is incompatible.
crewai 1.7.2 requires openai~=1.83.0, but you have openai 2.14.0 which is incompatible.


In [8]:
# Retrieve the API keys securely from Google Colab's user data
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')
serper_api_key = userdata.get('serper_api')

In [9]:
import os
# Set the API keys as environment variables for accessibility
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [10]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool
from pydantic import BaseModel

In [11]:
# Define the LLM
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.5)

# Data Inputs

In [12]:
brand_info = {
    "name": "Zero to Mastery",
    "description": "An online tech academy that offers comprehensive courses and a community for learners seeking in-demand software and tech skills.",
    "brand_values": "Quality education, community-driven growth, career-focused learning, and up-to-date tech stacks.",
    "ideal_partner": "A YouTube channel focused on programming, software engineering, or tech career advice with an audience of motivated learners."
}

In [13]:
# Define a single YouTube lead for demonstration
youtube_lead = {
    "name": "Theo - t3․gg",
    "channel_url": "https://www.youtube.com/@t3dotgg"
}

# Research

In [14]:
# Build a Research Agent
research_agent = Agent(
    role = "Influencer researcher",
    goal = f"Research about {youtube_lead['name']}'s channel {youtube_lead['channel_url']} to assess their suitability for {brand_info['name']}",
    backstory = f"You are an expert in Influencer partnerships that align with {brand_info['name']} and its respective values: {brand_info['brand_values']}",
    llm = llm,
    tools = [SerperDevTool(), ScrapeWebsiteTool(), YoutubeChannelSearchTool(), YoutubeVideoSearchTool()]
)

In [15]:
# Define the google research task
channel_research_task = Task(
    description = f"""
    Research the channel {youtube_lead['name']} at {youtube_lead['channel_url']}:
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with {brand_info['name']}""",
    expected_output = "A research report on the channel's key attributes and audience",
    agent = research_agent
)

In [16]:
# Define the research videos task
videos_research_task = Task(
    description = f"""
    Research the most popular videos from channel {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Summarise their topics and analyse the audiences.""",
    expected_output="A research report on the most popular videos and their topics",
    agent = research_agent)

In [17]:
# Define a sentiment on the channel via Google searches
sentiment_task = Task(
    description = f"""
    Conduct a sentiment analysis via Google Search on the channel {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Look for user feedbacks, resviews, or discussions that indicate general sentiment.
    Search for instance on reddit.""",
    expected_output= "A sentiment analysis report on the channel's",
    agent = research_agent)

# AI Scoring Agent

In [18]:
# Developping the Scoring Agent
scoring_agent =  Agent(
    role = "Partnership Scoring Agent",
    goal = f"""Score the partnership between {brand_info['name']} and {youtube_lead['name']}.
    Assess based on the {brand_info['brand_values']}""",
    backstory = f"You are an expert in Influencer Marking working for {brand_info['name']}",
    llm = llm,
)

In [19]:
# Create a Base Model Class to provide a score
class Score(BaseModel):
    score: int
    justification: str

In [20]:
# Define the Scoring Task
scoring_task = Task(
    description = f"""
    Score the partnership between {brand_info['name']} and {youtube_lead['name']}""",
    expected_output = "A score between 1 and 10 and justification for the scoring. An explanation of the score indicating what fits the brand and what does not",
    agent = scoring_agent,
    context = [channel_research_task, videos_research_task, sentiment_task],
    output_pydantic= Score
)


# Build the AI Crew

In [21]:
# Assemble the AI crew
crew = Crew(
    agents = [research_agent, scoring_agent],
    tasks = [channel_research_task, videos_research_task, sentiment_task, scoring_task],
    process = Process.sequential,
    verbose = True
)
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2d2a3226-219f-4234-9dc5-01c017226c4a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Influencer researcher                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Research the channel Theo - t3․gg at https://www.youtube.com/@t3dotgg:                                     │
│      1. Identify channel focus and content type.                                                                │
│      2. Determine approximate subscriber count and engagement.                                                  │
│      3. Check for audience alignment with Zero to Mastery                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Influencer researcher                                                                                   │
│                                                                                                                 │
│  Thought: I should start by searching for the channel "Theo - t3․gg" on YouTube to gather information about     │
│  its focus, content type, subscriber count, and engagement. This will help me understand if it aligns with      │
│  Zero to Mastery's values.                                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 96d30467-8a47-491e-a01b-4eadaf931fa8                                                                     │
│  Agent: Influencer researcher                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Partnership Scoring Agent                                                                               │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Score the partnership between Zero to Mastery and Theo - t3․gg                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Partnership Scoring Agent                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "score": 8,                                                                                                  │
│    "justification": "The partnership between Zero to Mastery and Theo - t3․gg is rated as an 8 out of 10.       │
│  Theo's channel aligns well with Zero to Mastery's focus on quality education, community-driven growth, and     │
│  career-focused learning in the field of technology and software development. The channel's emphasis on         │
│  software development, AI, and up-to-date tech stacks matches Zero to Mastery's goals of providing current and  │
│  relevant educational content. The significant subscriber count and high engagement levels demonstrate a        │
│  strong community presence, which is beneficial for community-driven growth. However, there are some areas for  │
│  improvement, such as making the content more concise and addressing communication challenges that some         │
│  viewers experience. Despite these areas for improvement, the overall alignment and potential for mutual        │
│  benefit make this partnership a strong fit."                                                                   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [22]:

import json
# Accessing the final outputs
channel_research_output = result.tasks_output[0].raw
videos_research_output = result.tasks_output[1].raw
sentiment_output = result.tasks_output[2].raw
scoring_output = result.tasks_output[3].raw

print("Channel Research Output:")
display(Markdown(channel_research_output))

print("\nVideos Research Output:")
display(Markdown(videos_research_output))

print("\nSentiment Analysis Output:")
display(Markdown(sentiment_output))

print("\nScoring Output:")
display(Markdown(scoring_output))

Channel Research Output:
Would you like to view your execution traces? [y/N] (20s timeout): 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cea0d4ce-22ab-4e39-a0d7-51b58fad571c                                                                     │
│  Agent: Partnership Scoring Agent                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

**Research Report on Theo - t3․gg YouTube Channel**

1. **Channel Focus and Content Type**:
   - Theo - t3․gg's channel focuses on software development, AI, and technology-related content. It covers topics such as coding practices, AI advancements, and tech stack discussions. The content is educational and often includes tutorials, insights, and personal experiences related to technology and software development.

2. **Subscriber Count and Engagement**:
   - As of the latest data, the channel has approximately 493,000 subscribers. The channel has a significant level of engagement, with total video views reaching over 88 million. The estimated monthly earnings range from $9,000 to $28,000, indicating strong viewer engagement and monetization potential. The average video length is around 32 minutes, suggesting in-depth content.

3. **Audience Alignment with Zero to Mastery**:
   - The channel's focus on quality education in software development, AI, and up-to-date tech stacks aligns well with Zero to Mastery's values of providing quality education and career-focused learning. The community-driven aspect is evident through active engagement metrics and the educational nature of the content, which could appeal to individuals seeking to enhance their skills in technology and software development.

Overall, Theo - t3․gg's YouTube channel appears to be a suitable candidate for partnership with Zero to Mastery, given its alignment with the educational and technological focus of the platform.


Videos Research Output:


**Research Report on Theo - t3․gg YouTube Channel**

1. **Channel Focus and Content Type**:
   - Theo - t3․gg's channel focuses on software development, AI, and technology-related content. It covers topics such as coding practices, AI advancements, and tech stack discussions. The content is educational and often includes tutorials, insights, and personal experiences related to technology and software development.

2. **Subscriber Count and Engagement**:
   - As of the latest data, the channel has approximately 493,000 subscribers. The channel has a significant level of engagement, with total video views reaching over 88 million. The estimated monthly earnings range from $9,000 to $28,000, indicating strong viewer engagement and monetization potential. The average video length is around 32 minutes, suggesting in-depth content.

3. **Popular Videos and Topics**:
   - One of the most popular videos is "2025: The year I stopped writing code," which discusses the impact of AI on coding practices. The video highlights the advancements in AI models and tools that have changed how developers write code, emphasizing the concept of "vibe coding" where AI tools handle much of the coding process. This video resonates with the audience interested in AI and its practical applications in software development.

4. **Audience Alignment with Zero to Mastery**:
   - The channel's focus on quality education in software development, AI, and up-to-date tech stacks aligns well with Zero to Mastery's values of providing quality education and career-focused learning. The community-driven aspect is evident through active engagement metrics and the educational nature of the content, which could appeal to individuals seeking to enhance their skills in technology and software development.

Overall, Theo - t3․gg's YouTube channel appears to be a suitable candidate for partnership with Zero to Mastery, given its alignment with the educational and technological focus of the platform.


Sentiment Analysis Output:


1. **Positive Sentiment**: 
   - Theo's content is appreciated for being genuine and engaging, especially for mid and senior engineers. His style is noted as unique among tech influencers, providing valuable insights into AI and technology.
   - Users have expressed enjoyment in his updates on the AI world and the detailed analysis of features and behaviors.

2. **Negative Sentiment**:
   - Some users find Theo's communication style challenging, making the content occasionally hard to follow.
   - There are criticisms regarding the length and depth of his content, with some users feeling that the videos could be more concise.

3. **Mixed Sentiment**:
   - While some users appreciate Theo's honesty and unique perspectives, others feel that his takes can be controversial or the "worst possible."
   - Theo's opinions on certain programming languages and practices, such as his dislike for Go or his views on unit testing, have sparked mixed reactions.

Overall, the sentiment towards Theo - t3․gg's YouTube channel is generally positive, with appreciation for the educational value and unique insights, but there are areas for improvement in communication and content delivery.


Scoring Output:


{
  "score": 8,
  "justification": "The partnership between Zero to Mastery and Theo - t3․gg is rated as an 8 out of 10. Theo's channel aligns well with Zero to Mastery's focus on quality education, community-driven growth, and career-focused learning in the field of technology and software development. The channel's emphasis on software development, AI, and up-to-date tech stacks matches Zero to Mastery's goals of providing current and relevant educational content. The significant subscriber count and high engagement levels demonstrate a strong community presence, which is beneficial for community-driven growth. However, there are some areas for improvement, such as making the content more concise and addressing communication challenges that some viewers experience. Despite these areas for improvement, the overall alignment and potential for mutual benefit make this partnership a strong fit."
}